In [1]:
%matplotlib inline

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib, collections, datetime, hashlib, os, re, time, sys
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 读取文件 

In [2]:
all_raw = pd.read_json("./hk_data/doh3/result/result_doh3.txt",lines=True)
location_raw = pd.read_json("./hk_data/doh3/location/all_doh3_location.txt", lines=True)
location_raw = pd.DataFrame(location_raw,columns = ['query','country','isp','org','as','countryCode'])
location_raw = location_raw.rename(columns={"query":"ip"})
all_raw = pd.merge(all_raw, location_raw, on=['ip'])


all_raw.drop_duplicates(subset=['ip'], keep='first', inplace=True)
all_raw

,ip,suffix,geth_3,posth_3,tlsVersion,success_flag,connError,raw_cert_chain,cert_level,certValid,...,ocsp_stapling_cert_status,ocsp_stapling_response_sig,hsts,expectct,hpkp,country,isp,org,as,countryCode
0,102.68.86.68,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,Good,True,max-age=63072000; includeSubDomains; preload,,,Kenya,VIDOLU Group Pty Ltd-Web4Africa,,AS327813 VIDOLU Group Pty Ltd-Web4Africa,KE
4,23.247.212.47,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,Good,True,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US
8,23.247.213.237,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,Good,True,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US
12,23.247.214.241,resolve,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,Good,True,max-age=63072000; includeSubDomains; preload,,,France,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",FR
16,23.247.215.17,,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,Good,True,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2283,8.8.8.8,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,False,,,,United States,Google LLC,Google Public DNS,AS15169 Google LLC,US
2284,113.161.194.80,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,False,,,,Vietnam,VietNam Post and Telecom Corporation,Vietnam Posts and Telecommunications Group,AS45899 VNPT Corp,VN
2285,103.137.62.18,dns-query,True,True,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUU5ak...,DV,True,...,,False,,,,Taiwan,"Pittqiao Network Information Co., Ltd.","Pittqiao Network Information Co., Ltd.","AS131642 Pittqiao Network Information Co.,Ltd.",TW
2286,84.221.19.44,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,False,,,,Italy,Tiscali Italia SpA,Tiscali Italia SpA,AS8612 Tiscali Italia S.P.A.,IT


In [3]:
all_raw.keys()

Index(['ip', 'suffix', 'geth_3', 'posth_3', 'tlsVersion', 'success_flag',
       'connError', 'raw_cert_chain', 'cert_level', 'certValid', 'caName',
       'time_valid', 'cert_lifetime', 'cert_error', 'ct_tls', 'sct_tls_valid',
       'sct_tls_log', 'ct_cert', 'sct_cert_valid', 'sct_cert_log', 'ct_ocsp',
       'sct_ocsp_valid', 'sct_ocsp_log', 'ct_valid', 'crLs', 'crl_server',
       'ocsp', 'ocspServer', 'ocspStapling', 'ocspMustStaple', 'respect_ms',
       'response_flag', 'crl_cert_status', 'crl_response_sig',
       'ocsp_cert_status', 'ocsp_response_sig', 'ocsp_stapling_cert_status',
       'ocsp_stapling_response_sig', 'hsts', 'expectct', 'hpkp', 'country',
       'isp', 'org', 'as', 'countryCode'],
      dtype='object')

In [4]:
all_raw['tlsVersion'].value_counts()
print("--------------------------")
all_raw['tlsVersion'].value_counts()/len(all_raw)*100

tls1_3    580
Name: tlsVersion, dtype: int64

--------------------------


tls1_3    100.0
Name: tlsVersion, dtype: float64

In [5]:
all_raw['certValid'].value_counts()
print("--------------------------")
all_raw['certValid'].value_counts()/len(all_raw)*100

True    580
Name: certValid, dtype: int64

--------------------------


True    100.0
Name: certValid, dtype: float64

In [6]:
all_raw['caName'].value_counts()
print("--------------------------")
all_raw['caName'].value_counts()/len(all_raw)*100

ZeroSSL                      572
Google Trust Services LLC      7
Let's Encrypt                  1
Name: caName, dtype: int64

--------------------------


ZeroSSL                      98.620690
Google Trust Services LLC     1.206897
Let's Encrypt                 0.172414
Name: caName, dtype: float64

In [7]:
all_raw['cert_lifetime'].value_counts()
print("--------------------------")
all_raw['cert_lifetime'].value_counts()/len(all_raw)*100

90.999988     571
83.999988       5
89.999988       3
365.999988      1
Name: cert_lifetime, dtype: int64

--------------------------


90.999988     98.448276
83.999988      0.862069
89.999988      0.517241
365.999988     0.172414
Name: cert_lifetime, dtype: float64

# CT & Revoke 

In [8]:
all_raw['ct_valid'].value_counts()
print("--------------------------")
all_raw['ct_valid'].value_counts()/len(all_raw)*100

True    580
Name: ct_valid, dtype: int64

--------------------------


True    100.0
Name: ct_valid, dtype: float64

In [9]:
have_ct_num = len(all_raw[(all_raw['ct_cert']==True) | (all_raw['ct_tls']==True)| (all_raw['ct_ocsp']==True)])
print(have_ct_num)
print(have_ct_num/len(all_raw)*100)

580
100.0


In [33]:
def list_convert(arg):
    new_list = []
    for i in arg:
        if type(i) is not list:
            new_list.append(i)
        else:
            new_list.extend(list_convert(i))
    return new_list

In [34]:
cert_log_list = []
cert_ct_raw = all_raw[all_raw['ct_cert']==True]
for index,value in cert_ct_raw["sct_cert_log"].iteritems():
    temp = value.split("#||#")
    cert_log_list.append(temp)
    
temp = list_convert(cert_log_list)
cert_log =pd.Series(temp)
print(cert_log.value_counts())   #null即该域名有cert传递的sct，但是certvalid都为false。或第一个为flase,第二个为true
print("-------------------------------")
print(cert_log.value_counts()/len(temp)*100)
print("-------------------------------")

google_log = 0
for index,value in cert_ct_raw["sct_cert_log"].iteritems():
    temp = value.split("#||#")
    for i in range(len(temp)):
        if "Google" in temp[i]:
            google_log = google_log + 1
            break
print(google_log, google_log/len(cert_ct_raw)*100)
print("-------------------------------")

google_log = 0
all_log = 0
for index,value in cert_log.value_counts().items():
    if "Google" in index:
        google_log = google_log + value
    if index != "":
        all_log = all_log + value
print(google_log, google_log/all_log*100)


Google 'Xenon2022' log         571
Cloudflare 'Nimbus2022' Log    571
Cloudflare 'Nimbus2023' Log      9
Google 'Argon2023' log           6
Google 'Xenon2023' log           4
dtype: int64
-------------------------------
Google 'Xenon2022' log         49.181740
Cloudflare 'Nimbus2022' Log    49.181740
Cloudflare 'Nimbus2023' Log     0.775194
Google 'Argon2023' log          0.516796
Google 'Xenon2023' log          0.344531
dtype: float64
-------------------------------
580 100.0
-------------------------------
581 50.043066322136085


## CT LOG 

In [10]:
ct_prase = pd.read_json("./hk_data/doh3/scan/result_ct_prase.txt",lines=True)
ct_prase

,ip,googleCertLog,appleCertLog
0,37.252.251.157,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
1,45.90.29.225,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2,45.90.29.44,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
3,23.247.213.109,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
4,23.247.212.231,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
...,...,...,...
2259,23.247.214.109,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2260,23.247.215.193,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2261,45.90.30.78,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2262,23.247.214.3,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...


In [11]:
all_raw = pd.merge(all_raw,ct_prase,on=['ip'],how='left')
all_raw.drop_duplicates(subset=['ip'], keep='first', inplace=True)
all_raw

,ip,suffix,geth_3,posth_3,tlsVersion,success_flag,connError,raw_cert_chain,cert_level,certValid,...,hsts,expectct,hpkp,country,isp,org,as,countryCode,googleCertLog,appleCertLog
0,102.68.86.68,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,max-age=63072000; includeSubDomains; preload,,,Kenya,VIDOLU Group Pty Ltd-Web4Africa,,AS327813 VIDOLU Group Pty Ltd-Web4Africa,KE,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
4,23.247.212.47,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
5,23.247.213.237,query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
9,23.247.214.241,resolve,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,max-age=63072000; includeSubDomains; preload,,,France,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",FR,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
13,23.247.215.17,,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUVJak...,DV,True,...,max-age=63072000; includeSubDomains; preload,,,United States,nextdns,"nextdns, Inc.","AS34939 nextdns, Inc.",US,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261,8.8.8.8,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,,,United States,Google LLC,Google Public DNS,AS15169 Google LLC,US,Google 'Argon2023' log###Cloudflare 'Nimbus202...,Google 'Argon2023' log###Cloudflare 'Nimbus202...
2263,113.161.194.80,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,,,Vietnam,VietNam Post and Telecom Corporation,Vietnam Posts and Telecommunications Group,AS45899 VNPT Corp,VN,NaN,NaN
2264,103.137.62.18,dns-query,True,True,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUU5ak...,DV,True,...,,,,Taiwan,"Pittqiao Network Information Co., Ltd.","Pittqiao Network Information Co., Ltd.","AS131642 Pittqiao Network Information Co.,Ltd.",TW,Google 'Xenon2023' log###Cloudflare 'Nimbus202...,Google 'Xenon2023' log###Cloudflare 'Nimbus202...
2265,84.221.19.44,dns-query,True,False,tls1_3,True,,LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSUY0ek...,DV,True,...,,,,Italy,Tiscali Italia SpA,Tiscali Italia SpA,AS8612 Tiscali Italia S.P.A.,IT,NaN,NaN


In [12]:
all_raw["googleCertLog"] = all_raw["googleCertLog"].fillna("")
all_raw["appleCertLog"] = all_raw["appleCertLog"].fillna("")
all_raw["sct_tls_log"] = all_raw["sct_tls_log"].fillna("")
all_raw["sct_ocsp_log"] = all_raw["sct_ocsp_log"].fillna("")

In [13]:
ALL_ct_log = []
log_file = open("./log_list.json", "r")
log_data = json.loads(log_file.read())
for i in range(len(log_data['operators'])):
    operators = log_data['operators'][i]

    for j in range(len(operators['logs'])):
        logs = operators['logs'][j]
        ALL_ct_log.append(logs['description'])

In [14]:

import datetime
import json
import base64
from OpenSSL import crypto

def get_ct_log(cert_log,tls_log,ocsp_log):
    cert_log_list = cert_log.split("###")
    tls_log_list = tls_log.split("#||#")
    ocsp_log_list = ocsp_log.split("#||#")
    
    result_ct_log = []

    for i in range(len(cert_log_list)):
        log = cert_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)
    
    for i in range(len(ocsp_log_list)):
        log = ocsp_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)
    
    for i in range(len(tls_log_list)):
        log = tls_log_list[i]
#         if log in ALL_ct_log and log not in result_ct_log:
        if log not in result_ct_log:
            result_ct_log.append(log)

    return result_ct_log

def check_google_old(log_list):
    google_flag = False
    none_google_flag = False

    for i in range(len(log_list)):
        if 'Google' in log_list[i]:
            google_flag = True
        else:
            none_google_flag = True

    if google_flag and none_google_flag:
        return True
    else:
        return False

def check_google_policy(data):
    raw_cert_chain = data['raw_cert_chain']
    cert_life_time = data['cert_lifetime']
    googleCertLog = data['googleCertLog']
    tls_log = data['sct_tls_log']
    ocsp_log = data['sct_ocsp_log']
    have_ct = data['ct_cert'] or data['ct_ocsp'] or data['ct_tls']

    if raw_cert_chain != "" and have_ct == True and type(googleCertLog) != float :
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        notBefore = datetime.datetime.strptime(cert_body.get_notBefore().decode(), "%Y%m%d%H%M%SZ")
        notAfter = datetime.datetime.strptime(cert_body.get_notAfter().decode(), "%Y%m%d%H%M%SZ")

        google_time = datetime.datetime.strptime('2022-04-15', "%Y-%m-%d")
        ct_log_list = get_ct_log(googleCertLog,tls_log,ocsp_log)

        if notBefore > google_time:
            if cert_life_time <=180:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=180;log<2"
            else:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time>180;log<3"
        else:
            google_none_google = check_google_old(ct_log_list)
            if google_none_google:
                if cert_life_time <= 450:
                    if len(ct_log_list) >= 2:
                        return "success"
                    else:
                        return "old;time<=450;log<2"

                elif cert_life_time <= 810:
                    if len(ct_log_list) >= 3:
                        return "success"
                    else:
                        return "old;time<=810;log<3"

                elif cert_life_time <= 1170:
                    if len(ct_log_list) >= 4:
                        return "success"
                    else:
                        return "old;time<=1170;log<4"

                elif cert_life_time > 1170:
                    if len(ct_log_list) >= 5:
                        return "success"
                    else:
                        return "old;time>1170;log<5"

            else:
                return "old;google_none_google"

    else:
        return "no support ct"
    

def check_apple_policy(data):
    raw_cert_chain = data['raw_cert_chain']
    cert_life_time = data['cert_lifetime']
    appleCertLog = data['appleCertLog']
    tls_log = data['sct_tls_log']
    ocsp_log = data['sct_ocsp_log']
    have_ct = data['ct_cert'] or data['ct_ocsp'] or data['ct_tls']
    
    if raw_cert_chain != "" and have_ct == True and type(appleCertLog) != float :
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        notBefore = datetime.datetime.strptime(cert_body.get_notBefore().decode(), "%Y%m%d%H%M%SZ")
        notAfter = datetime.datetime.strptime(cert_body.get_notAfter().decode(), "%Y%m%d%H%M%SZ")

        apple_time = datetime.datetime.strptime('2021-04-21', "%Y-%m-%d")
        ct_log_list = get_ct_log(appleCertLog,tls_log,ocsp_log)
        
        if notBefore > apple_time:
            if cert_life_time <=180:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=180;log<2"
            else:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time>180;log<3"
        else:
            if cert_life_time <=450:
                if len(ct_log_list) >= 2:
                    return "success"
                else:
                    return "new;time<=450;log<2"
            elif cert_life_time <=810:
                if len(ct_log_list) >= 3:
                    return "success"
                else:
                    return "new;time<=810;log<3"
            
            elif cert_life_time <=1170:
                if len(ct_log_list) >= 4:
                    return "success"
                else:
                    return "new;time<=1170;log<4"
                
            else:
                if len(ct_log_list) >= 5:
                    return "success"
                else:
                    return "new;time>1170;log<5"
    else:
        return "no support ct"
        
    
    
def check_subject(data):
    raw_cert_chain = data['raw_cert_chain']

    if raw_cert_chain != "":
        leaf_cert = raw_cert_chain.split("###")[0]
        pem = base64.b64decode(leaf_cert)
        cert_body = crypto.load_certificate(crypto.FILETYPE_PEM, pem)  # 从类型为type编码的字符串缓冲区加载证书 (X509)
        cn =cert_body.get_subject().CN
        org =cert_body.get_subject().O
        ou =cert_body.get_subject().OU
        result = ""

        if cn != None:
            result = result + cn + " "
        
        if org != None:
            result = result + org + " "
        
        if ou != None:
            result = result + ou + " "
        
        result = result.replace(",", " ")
        result = result.replace("@", " ")
        result = result.replace(";", " ")
        result = result.replace("*", "")
        result = result.replace("(", "")
        result = result.replace(")", "")
        return result
    else:
        if type(data['org']) == str and data['org']!= "":
            result = data['org']
            result = result.replace(",", " ")
            result = result.replace("@", " ")
            result = result.replace(";", " ")
            result = result.replace("*", "")
            result = result.replace("(", "")
            result = result.replace(")", "")
            
            return result
        else:
            return "unknown"
        
        

In [15]:
all_raw = all_raw.assign(google_ct_policy=all_raw.apply(lambda x: check_google_policy(x), axis=1))

print(all_raw['google_ct_policy'].value_counts())
print("---------------------------------")
print(all_raw['google_ct_policy'].value_counts()/len(all_raw)*100)

success                577
new;time<=180;log<2      3
Name: google_ct_policy, dtype: int64
---------------------------------
success                99.482759
new;time<=180;log<2     0.517241
Name: google_ct_policy, dtype: float64


In [16]:
all_raw = all_raw.assign(apple_ct_policy=all_raw.apply(lambda x: check_apple_policy(x), axis=1))

print(all_raw['apple_ct_policy'].value_counts())
print("---------------------------------")
print(all_raw['apple_ct_policy'].value_counts()/len(all_raw)*100)

success                577
new;time<=180;log<2      3
Name: apple_ct_policy, dtype: int64
---------------------------------
success                99.482759
new;time<=180;log<2     0.517241
Name: apple_ct_policy, dtype: float64


In [41]:
temp = all_raw[all_raw['google_ct_policy'] == 'new;time<=180;log<2']
for i in temp['sct_cert_log']:
    print(i)


Google 'Xenon2022' log#||#Cloudflare 'Nimbus2022' Log
Google 'Argon2023' log#||#Cloudflare 'Nimbus2023' Log
Google 'Argon2023' log#||#Cloudflare 'Nimbus2023' Log


In [17]:
all_raw = all_raw.assign(subject=all_raw.apply(lambda x: check_subject(x), axis=1))
print(all_raw['subject'].value_counts())
print("---------------------------------")
print(all_raw['subject'].value_counts()/len(all_raw)*100)

dns.nextdns.io      570
dns.google            5
.qvq.network          2
.squallatf.info       1
35.190.81.130         1
101.33.33.234         1
Name: subject, dtype: int64
---------------------------------
dns.nextdns.io      98.275862
dns.google           0.862069
.qvq.network         0.344828
.squallatf.info      0.172414
35.190.81.130        0.172414
101.33.33.234        0.172414
Name: subject, dtype: float64


In [18]:
from drain3.drain import Drain, LogCluster
model = Drain(param_str="(.*)")

for entry in list(all_raw['subject']):
    cluster, change_type = model.add_log_message(entry)

print(model.clusters_counter)
template_list = []
for i in range(model.clusters_counter):
    result = str(model.id_to_cluster[i+1]).split(":")[2][1:]
    template_list.append(result)
#     print(model.id_to_cluster[i+1])
# print(template_list)

6


In [19]:
import re
def group_subject(data):
    subject = data['subject']
    org = data['org']
    org = org.replace(",", " ")
    org = org.replace("@", " ")
    org = org.replace(";", " ")
    org = org.replace("*", "")
    org = org.replace("(", "")
    org = org.replace(")", "")
    result = ""
    for i in range(len(template_list)):
        template = template_list[i]
#         print(template)
#         print(subject)
#         print(org)
#         print("--------")
        if subject == template or org == template:
            result = template
            break
        else:
            if len(re.findall(template,subject)) != 0 or len(re.findall(template,org)) != 0:
                result = template
                break
    if result == "": 
        return subject
    else:
        return result


In [20]:
all_raw = all_raw.assign(group=all_raw.apply(lambda x: group_subject(x), axis=1))

print(all_raw['group'].value_counts())
print("---------------------------------")
print(all_raw['group'].value_counts()/len(all_raw)*100)

dns.nextdns.io     570
dns.google           5
.qvq.network         2
.squallatf.info      1
35.190.81.130        1
101.33.33.234        1
Name: group, dtype: int64
---------------------------------
dns.nextdns.io     98.275862
dns.google          0.862069
.qvq.network        0.344828
.squallatf.info     0.172414
35.190.81.130       0.172414
101.33.33.234       0.172414
Name: group, dtype: float64


In [28]:
# temp = all_raw[(all_raw['group']=="nextdns  Inc") | (all_raw['group']=="dns.nextdns.io")]
temp = all_raw[all_raw['group']=='dns.google']
temp.country.value_counts()/len(temp)*100

Vietnam          40.0
Brazil           20.0
United States    20.0
Italy            20.0
Name: country, dtype: float64

## Revoke 

In [22]:
all_raw['crLs'].value_counts()
print("--------------------------")
all_raw['crLs'].value_counts()/len(all_raw)*100

False    573
True       7
Name: crLs, dtype: int64

--------------------------


False    98.793103
True      1.206897
Name: crLs, dtype: float64

In [23]:
all_raw['ocsp'].value_counts()
print("--------------------------")
all_raw['ocsp'].value_counts()/len(all_raw)*100

True    580
Name: ocsp, dtype: int64

--------------------------


True    100.0
Name: ocsp, dtype: float64

In [24]:
all_raw['ocspStapling'].value_counts()
print("--------------------------")
all_raw['ocspStapling'].value_counts()/len(all_raw)*100

True     571
False      9
Name: ocspStapling, dtype: int64

--------------------------


True     98.448276
False     1.551724
Name: ocspStapling, dtype: float64

In [25]:
all_raw['ocspMustStaple'].value_counts()
print("--------------------------")
all_raw['ocspMustStaple'].value_counts()/len(all_raw)*100

True     570
False     10
Name: ocspMustStaple, dtype: int64

--------------------------


True     98.275862
False     1.724138
Name: ocspMustStaple, dtype: float64

In [26]:
all_raw['respect_ms'].value_counts()
print("--------------------------")
all_raw['respect_ms'].value_counts()/len(all_raw)*100

True     570
False     10
Name: respect_ms, dtype: int64

--------------------------


True     98.275862
False     1.724138
Name: respect_ms, dtype: float64

# HTTP 版本 

In [45]:
result_doh = pd.read_csv("./hk_data/doh3/result_doh3.txt")
result_doh

,ip,getH1,getH2,postH1,postH2
0,101.33.33.234,True,True,True,True
1,103.137.62.19,True,True,True,True
2,103.137.62.18,True,True,True,True
3,194.156.163.172,True,True,True,True
4,203.76.245.200,True,True,True,True
...,...,...,...,...,...
569,45.90.31.93,True,True,True,True
570,45.90.31.91,True,True,True,True
571,45.150.243.134,True,True,True,True
572,68.67.75.93,True,True,True,True


In [46]:
temp = result_doh[(result_doh['getH1']==True)|(result_doh['postH1']==True)]
print(len(temp)/len(result_doh)*100)

100.0


In [47]:
temp = result_doh[(result_doh['getH2']==True)|(result_doh['postH2']==True)]
print(len(temp)/len(result_doh)*100)

100.0
